## Lesson-01 Assignment

#### 今天是2019年9月28日，今天世界上又多了一名AI工程师 :) 

`各位同学大家好，欢迎各位开始学习我们的人工智能课程。这门课程假设大家不具备机器学习和人工智能的知识，但是希望大家具备初级的Python编程能力。根据往期同学的实际反馈，我们课程的完结之后 能力能够超过80%的计算机人工智能/深度学习方向的硕士生的能力。`

## 本次作业的内容

#### 1. 复现课堂代码

在本部分，你需要参照我们给大家的GitHub地址里边的课堂代码，结合课堂内容，复现内容。

### 1-1 句子生成器

In [3]:
#coding:utf-8
import random

simple_grammar = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>  蓝色的 | 好看的 | 小小的
"""

def create_grammar(grammar_str,split='=>',line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip():
            continue
        exp,stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar

def generate(gram,target):
    if target not in gram:
        return target
    expaned = [generate(gram,t) for t in random.choice(gram[target])]
    return ''.join([e for e in expaned if e != 'null'])

In [4]:
example_grammar = create_grammar(simple_grammar)
example_grammar

{'sentence': [['noun_phrase', 'verb_phrase']],
 'noun_phrase': [['Article', 'Adj*', 'noun']],
 'Adj*': [['null'], ['Adj', 'Adj*']],
 'verb_phrase': [['verb', 'noun_phrase']],
 'Article': [['一个'], ['这个']],
 'noun': [['女人'], ['篮球'], ['桌子'], ['小猫']],
 'verb': [['看着'], ['坐在'], ['听着'], ['看见']],
 'Adj': [['蓝色的'], ['好看的'], ['小小的']]}

In [5]:
generation = generate(example_grammar,target='sentence')
generation

'一个好看的桌子坐在一个女人'

In [6]:
#在西部世界里，一个”人类“的语言可以定义为：

human = """
human = 自己 寻找 活动
自己 = 我 | 俺 | 我们 
寻找 = 找找 | 想找点 
活动 = 乐子 | 玩的
"""


#一个“接待员”的语言可以定义为

host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = null
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？
"""

for i in range(10):
    print(generate(gram=create_grammar(host,split='='),target='host'))

先生,你好我是51号,您需要赌博吗？
先生,您好我是6号,请问你要打牌吗？
你好我是2号,请问你要打猎吗？
你好我是2号,请问你要喝酒吗？
先生,您好我是71号,请问你要赌博吗？
女士,你好我是7号,请问你要喝酒吗？
您好我是17号,您需要赌博吗？
小朋友,你好我是8号,您需要喝酒吗？
您好我是21号,您需要喝酒吗？
小朋友,您好我是69号,请问你要打猎吗？


In [7]:
programming = """
stmt => if_exp | while_exp | assignment 
assignment => var = var
if_exp => if ( var ) { /n .... stmt }
while_exp=> while ( var ) { /n .... stmt }
var => chars number
chars => char | char char
char => student | name | info  | database | course
number => 1 | 2 | 3
"""
print(generate(gram=create_grammar(programming),target='stmt'))

if(course3){/n....while(database2){/n....while(database3){/n....if(info1){/n....while(coursecourse1){/n....while(name2){/n....while(info3){/n....database3=studentinfo1}}}}}}}


### 1-2 语言模型

In [9]:
import jieba

def cut(string):
    return list(jieba.cut(string))

In [11]:
TOKEN = []
f = open('article_9k.txt')
while True:
    line = f.readline()
    if line:
        TOKEN += cut(line.strip())
    else:
        break
f.close()

len(TOKEN)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.562 seconds.
Prefix dict has been built succesfully.


17528643

In [12]:
from collections import Counter
words_count = Counter(TOKEN)
words_count.most_common(10)

[('的', 703716),
 ('n', 382020),
 ('在', 263597),
 ('月', 189330),
 ('日', 166300),
 ('新华社', 142462),
 ('和', 134061),
 ('年', 123106),
 ('了', 121938),
 ('是', 100909)]

In [13]:
# 进行拉普拉斯平滑
def prob_1(word):
    if word in words_count:
        return (words_count[word]+1) / (len(TOKEN)+len(words_count))
    else:
        return 1 / (len(TOKEN)+len(words_count))

In [14]:
prob_1('我们')

0.0011224969842337567

In [15]:
TOKEN = [str(t) for t in TOKEN]
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]
words_count_2 = Counter(TOKEN_2_GRAM)

In [20]:
def prob_2(word1,word2):
    if word1+word2 in words_count_2:
        prob_word12 = (words_count_2[word1+word2]+1) / (words_count[word1]+len(words_count))
        return prob_word12
    else:
        return 1 / (words_count[word1]+len(words_count))
    
def get_prob(sentence):
    words = cut(sentence)
    sent_pro = 1
    for i,word in enumerate(words[:-1]):
        next_word = words[i+1]
        sent_pro *= prob_2(word,next_word)
    return sent_pro

In [21]:
prob_2('我们','在')

0.0018357435272160047

In [22]:
get_prob("小明今天抽奖抽到一台苹果手机")

1.0796730303614397e-31

In [23]:
get_prob("小明抽奖抽到今天一架波音飞机")

8.81477284477152e-31

In [25]:
for sent in [generate(gram=example_grammar,target='sentence') for i in range(10)]:
    print('sentence:{} with Prb:{}'.format(sent,get_prob(sent)))

sentence:一个篮球坐在这个女人 with Prb:1.884023229281095e-21
sentence:这个蓝色的好看的小猫看见一个蓝色的小猫 with Prb:3.537982297032393e-47
sentence:一个女人听着一个小小的桌子 with Prb:3.0305137454732025e-27
sentence:这个好看的桌子坐在这个桌子 with Prb:3.435634786827028e-31
sentence:这个蓝色的女人看见一个篮球 with Prb:3.042360430093644e-29
sentence:一个小猫看见这个蓝色的蓝色的好看的蓝色的小猫 with Prb:6.862735926393314e-55
sentence:一个桌子看见一个蓝色的篮球 with Prb:2.7325307494049814e-29
sentence:这个女人看着这个桌子 with Prb:6.5189884116367324e-21
sentence:一个好看的桌子看见一个蓝色的桌子 with Prb:9.85939666811117e-39
sentence:一个蓝色的篮球坐在这个小小的小猫 with Prb:2.0525119273845165e-34


In [26]:
need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "今晚我去吃火锅 今晚火锅去吃我",
    "洋葱奶昔来一杯 养乐多绿来一杯"
]

In [27]:
for s in need_compared:
    s1,s2 = s.split()
    p1,p2 = get_prob(s1),get_prob(s2)
    better = s1 if p1 > p2 else s2
    print('{} is more possible'.format(better))
    print('-'*4 + '{} with probability {}'.format(s1,p1))
    print('-'*4 + '{} with probability {}'.format(s2,p2))

今天晚上请你吃大餐，我们一起吃日料 is more possible
----今天晚上请你吃大餐，我们一起吃日料 with probability 2.797138507178689e-48
----明天晚上请你吃大餐，我们一起吃苹果 with probability 2.3492811814369464e-48
真是一只好看的小猫 is more possible
----真事一只好看的小猫 with probability 1.4837901657038567e-25
----真是一只好看的小猫 with probability 4.0935523053517363e-20
今晚我去吃火锅 is more possible
----今晚我去吃火锅 with probability 1.2415668614088164e-14
----今晚火锅去吃我 with probability 1.2042860836863862e-20
养乐多绿来一杯 is more possible
----洋葱奶昔来一杯 with probability 9.390003594400455e-17
----养乐多绿来一杯 with probability 1.3355686938171338e-11


#### 2. 请回答以下问题

回答以下问题，并将问题发送至 minchuian.gao@gmail.com 中：
```
    2.1. what do you want to acquire in this course？
    2.2. what problems do you want to solve？
    2.3. what’s the advantages you have to finish you goal?
    2.4. what’s the disadvantages you need to overcome to finish you goal?
    2.5. How will you plan to study in this course period?
```

#### 3. 如何提交
代码 + 此 jupyter 相关，提交至自己的 github 中(**所以请务必把GitHub按照班主任要求录入在Trello中**)；
第2问，请提交至minchuian.gao@gmail.com邮箱。
#### 4. 作业截止时间
此次作业截止时间为 2019.10.8日

#### 5. 完成以下问答和编程练习

>

## 基础理论部分

#### 0. Can you come up out 3 sceneraies which use AI methods? 

Ans: 

{1 chatbot; 
 2 neural machine translation; 
 3 knowledge graph.}

#### 1. How do we use Github; Why do we use Jupyter and Pycharm;

Ans: 

(1) We use github because it can help us manage code and we can find many open source projects there.

(2) We use Jupyter because it is convenient for us to take notes and make a demonstration of program.

(3) We use Pycharm because it fits development projects of large scale.

#### 2. What's the Probability Model?

Ans:

(1) Probability model is a mathematical representation of a random phenomenon. It is defined by its sample space, events within the sample space, and probabilities associated with each event.

(2) The sample sapce S for a probability model is the set of all possible outcomes.

(3) An event A is a subset of the sample space S.

(4) A probability is a numerical value assigned to a given event A.

#### 3. Can you came up with some sceneraies at which we could use Probability Model?

Ans:

(1) Imaging a sceneray that passengers queue to buy train tickets to travel home for the spring festival holiday.Within time of t,the number of passengers obeys the **poisson distribution**. And the interval of successive passengers obeys a **negative exponential distribution**.

(2) In natural language processing, we can use probability model to solve **part of speech tagging problem** such as CRF. IF x represent word and y represent part of speech, then the probability of word x being a certain part of speech is p(y|x). 

#### 4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

Ans:

(1) By using probability, we can solve many problems which are highly uncertain because we can define a mathematical representation for a certain random phenomenon.Then, we can solve many problems by calculating the probability or ,more specifically, by ranking probabilities.

(2) I think there are two main diffficult points: firstly,there are so many possible answers that our program can't cover all situations; secondly, our program may can't handle some cases such as a sentence is not a grammatical sentence.

#### 5. What's the Language Model;

Ans:

1 The goal of Language Model (LM) is to calculate the probability of a sentence of sequences of words: $$p(W)=p(w_1,w_2,...,w_n)$$and can be used to find the probability of the next word in the sentence such as:$$p(w_5|w_1,w_2,w_3,w_4)$$

2 Language Model (LM) ban be classified into two categories: count-based and continuous-space LM. The count-based methods usually involve making an n-th order Markov assumption and estimating n-gram probabilities via counting and smoothing. In the other hand, Continuous-space LM such as Neural Language Model (NLM) solves the problem of data sparsity of the n-gram model, by representing words as vectors(word embeddings) ans using them as inputs to NLM.

#### 6. Can you came up with some sceneraies at which we could use Language Model?


Ans:

Language Model are useful in a broad range of applications, the most obvious perhaps being speech recognition and machine translation. For example, in speech recognition the language model is combined with an acoustic model that models the pronunciation of different words: one way is that the acoustic model generates a large numbers of candidate sentences; then the language model is used to calculate probabilities and reorder them based on how likely they are to be a sentence in the language.

#### 7. What's the 1-gram language model;

Ans:

In general cases, the formula of language model is as follows:$$p(w_1,w_2,...,w_n)=p(w_1)p(w_2|w_1)...p(w_n|w_1,w_2,...,w_{n-1})=\prod_ip(w_i|w_1,w_2,...,w_{i-1})$$The chain rule applied to compute the joined probability of words in a sequence.

However, the general solution need too much computation, therefore we can use conditional independence hypothesis or markov assumption to simply the estimation.

1-gram language model is based on complete conditional independence hypothesis,and the formula is as follows:$$p(w_1,w_2,...,w_n)=\prod_{i=1}^np(w_i)$$ Thus each word is chosen completely of other words in the sentence.

#### 8. What's the disadvantages and advantages of 1-gram language model;

Ans:

The most advantage of 1-gram language model is that this model can simplify the estimation of the probability distribution because of the conditional independence hypothesis,thus there are fewer parameters to calculate.

The disadvantage of 1-gram language model is that the assumption is arguably quite strong, and linguistically naive. 1-gram language model ignore the order between words in a sentence, so two different sentences with the same words may have the same probabilities if the only difference between them is the order between words. What's worse, an ungrammatical sentence may acquire a high probability.

#### 9. What't the 2-gram models;

Ans:

The 2-gram language model is a first-order Markov process,which considerably simplifies the model as follows:$$p(w_1,w_2,...,w_n)=\prod_ip(w_i|w_1,w_2,...,w_{i-1})\approx\prod_ip(w_i|w_{i-1})$$ The formula after the first equal sign is exact: by chain rule of probabilities. 

However,the second formula is not exact because we have assumed that the identiy of the i'th word $w_i$ in the sequence depends only on the identity of the previous word $w_{i-1}$.This is a first-order Markov assumption.

As you can see,2-gram language model is just this formula:$$p(w_1,w_2,...,w_n)\approx\prod_ip(w_i|w_{i-1})$$.

## 编程实践部分

#### 1. 设计你自己的句子生成器

如何生成句子是一个很经典的问题，从1940s开始，图灵提出机器智能的时候，就使用的是人类能不能流畅和计算机进行对话。和计算机对话的一个前提是，计算机能够生成语言。

计算机如何能生成语言是一个经典但是又很复杂的问题。 我们课程上为大家介绍的是一种基于规则（Rule Based）的生成方法。该方法虽然提出的时间早，但是现在依然在很多地方能够大显身手。值得说明的是，现在很多很实用的算法，都是很久之前提出的，例如，二分查找提出与1940s, Dijstra算法提出于1960s 等等。

在著名的电视剧，电影《西部世界》中，这些机器人们语言生成的方法就是使用的SyntaxTree生成语言的方法。

> 
>

![WstWorld](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1569578233461&di=4adfa7597fb380e7cc0e67190bbd7605&imgtype=0&src=http%3A%2F%2Fs1.sinaimg.cn%2Flarge%2F006eYYfyzy76cmpG3Yb1f)

> 
>

在这一部分，需要各位同学首先定义自己的语言。 大家可以先想一个应用场景，然后在这个场景下，定义语法。例如：

在西部世界里，一个”人类“的语言可以定义为：
``` 
human = """
human = 自己 寻找 活动
自己 = 我 | 俺 | 我们 
寻找 = 看看 | 找找 | 想找点
活动 = 乐子 | 玩的
"""
```

一个“接待员”的语言可以定义为
```
host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = 耍一耍 | 玩一玩
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？"""

```




请定义你自己的语法: 

第一个语法：

In [28]:
PUA = '''
PUA = 称谓 评价 自我介绍 询问 活动 结尾
称谓 = 女性称谓 ，
女性称谓 = 美女 | 靓女 | 小妹妹 | 小可爱 | 小仙女 | 小魔女 | 小姐姐
评价 = 你的 特征 形容 | 评价 你的 特征 形容
特征 = 部位 | 穿着
部位 = 眼睛 | 鼻子 | 美人痣
穿着 = 裙子 | 高跟鞋 | 耳环 | 汉服
形容 = 真好看 | 真美 | 真漂亮
自我介绍 = 。我是 职业 ，
职业 = 律师 | 医生 | 挨踢男 | 金融民工
询问 = 请问你能 | 你想 | 你愿意
活动 = 和我跳一支 舞 | 和我喝一杯 酒
舞 = trouble maker | 小苹果 | tango | 桑巴
酒 = 桃花酒 | 江小白 | 鸡尾酒 | 伏特加 | 黑啤
结尾 = 吗？
'''

In [29]:
def create_grammar(grammar_str,split='=',line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip():
            continue
        exp,stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar

In [30]:
pua_grammar = create_grammar(PUA)
pua_grammar

{'PUA': [['称谓', '评价', '自我介绍', '询问', '活动', '结尾']],
 '称谓': [['女性称谓', '，']],
 '女性称谓': [['美女'], ['靓女'], ['小妹妹'], ['小可爱'], ['小仙女'], ['小魔女'], ['小姐姐']],
 '评价': [['你的', '特征', '形容'], ['评价', '你的', '特征', '形容']],
 '特征': [['部位'], ['穿着']],
 '部位': [['眼睛'], ['鼻子'], ['美人痣']],
 '穿着': [['裙子'], ['高跟鞋'], ['耳环'], ['汉服']],
 '形容': [['真好看'], ['真美'], ['真漂亮']],
 '自我介绍': [['。我是', '职业', '，']],
 '职业': [['律师'], ['医生'], ['挨踢男'], ['金融民工']],
 '询问': [['请问你能'], ['你想'], ['你愿意']],
 '活动': [['和我跳一支', '舞'], ['和我喝一杯', '酒']],
 '舞': [['trouble', 'maker'], ['小苹果'], ['tango'], ['桑巴']],
 '酒': [['桃花酒'], ['江小白'], ['鸡尾酒'], ['伏特加'], ['黑啤']],
 '结尾': [['吗？']]}

In [31]:
import random

def generate(gram,target):
    if target not in gram:
        return target
    
    expaned = [generate(gram,i) for i in random.choice(gram[target])]
    return ''.join(expaned)
print(generate(pua_grammar,target='PUA'))

小妹妹，你的裙子真好看你的高跟鞋真漂亮。我是金融民工，你想和我跳一支troublemaker吗？


In [32]:
def generate_n(gram,target,n=10):
    for i in range(n):
        print(generate(gram,target),'\n')

In [11]:
generate_n(pua_grammar,target='PUA',n=10)

小可爱，你的裙子真好看。我是医生，请问你能和我跳一支小苹果吗？ 

靓女，你的眼睛真漂亮。我是医生，你愿意和我跳一支troublemaker吗？ 

小魔女，你的鼻子真好看你的美人痣真漂亮你的汉服真好看你的鼻子真漂亮。我是挨踢男，你愿意和我跳一支troublemaker吗？ 

小妹妹，你的美人痣真漂亮你的汉服真好看你的汉服真好看。我是金融民工，你愿意和我跳一支tango吗？ 

小仙女，你的耳环真漂亮你的裙子真美。我是金融民工，请问你能和我喝一杯鸡尾酒吗？ 

小魔女，你的鼻子真美你的高跟鞋真美。我是金融民工，你愿意和我跳一支小苹果吗？ 

小姐姐，你的眼睛真美。我是医生，你想和我跳一支troublemaker吗？ 

小妹妹，你的耳环真漂亮你的美人痣真美你的耳环真美。我是金融民工，你想和我喝一杯伏特加吗？ 

小妹妹，你的眼睛真好看。我是挨踢男，请问你能和我跳一支小苹果吗？ 

靓女，你的鼻子真好看你的耳环真好看。我是金融民工，请问你能和我跳一支桑巴吗？ 



第二个语法：

In [33]:
fans = '''
fans => 反应 明星 动作 感受 请求 结尾
反应 => 尖叫 ，
尖叫 => 哇 | 天呐 | 啊 | Oh_my_god | 尖叫 啊
明星 => 高明权老师 | 王静妍老师 | 邓紫棋 | 周杰伦
动作 => 在那里 行为 ，
行为 => 直播 | 上课 | 唱歌 | 跳舞
感受 => 感受 我要成仙了 | 我太开心了 | 我要疯了
请求 => 。我要过去 活动
活动 => 合照 | 签名 | 握手
结尾 => 。
'''

In [34]:
fans_grammar = create_grammar(fans,split='=>')
fans_grammar

{'fans': [['反应', '明星', '动作', '感受', '请求', '结尾']],
 '反应': [['尖叫', '，']],
 '尖叫': [['哇'], ['天呐'], ['啊'], ['Oh_my_god'], ['尖叫', '啊']],
 '明星': [['高明权老师'], ['王静妍老师'], ['邓紫棋'], ['周杰伦']],
 '动作': [['在那里', '行为', '，']],
 '行为': [['直播'], ['上课'], ['唱歌'], ['跳舞']],
 '感受': [['感受', '我要成仙了'], ['我太开心了'], ['我要疯了']],
 '请求': [['。我要过去', '活动']],
 '活动': [['合照'], ['签名'], ['握手']],
 '结尾': [['。']]}

In [35]:
print(generate(fans_grammar,target='fans'))

啊，高明权老师在那里跳舞，我太开心了。我要过去签名。


In [36]:
generate_n(fans_grammar,target='fans',n=10)

哇啊啊啊，周杰伦在那里跳舞，我要疯了。我要过去握手。 

Oh_my_god，邓紫棋在那里跳舞，我要疯了我要成仙了。我要过去握手。 

啊，邓紫棋在那里跳舞，我太开心了。我要过去合照。 

Oh_my_god，高明权老师在那里上课，我要疯了我要成仙了。我要过去握手。 

啊，周杰伦在那里跳舞，我要疯了。我要过去握手。 

天呐，高明权老师在那里上课，我太开心了我要成仙了我要成仙了我要成仙了。我要过去合照。 

啊，王静妍老师在那里跳舞，我要疯了。我要过去签名。 

天呐，王静妍老师在那里直播，我太开心了我要成仙了我要成仙了。我要过去签名。 

哇，周杰伦在那里上课，我要疯了。我要过去握手。 

哇啊，邓紫棋在那里直播，我太开心了。我要过去签名。 



>

#### 2. 使用新数据源完成语言模型的训练

按照我们上文中定义的`prob_2`函数，我们更换一个文本数据源，获得新的Language Model:

1. 下载文本数据集（你可以在以下数据集中任选一个，也可以两个都使用）
    + 可选数据集1，保险行业问询对话集： https://github.com/Computing-Intelligence/insuranceqa-corpus-zh/raw/release/corpus/pool/train.txt.gz
    + 可选数据集2：豆瓣评论数据集：https://github.com/Computing-Intelligence/datasource/raw/master/movie_comments.csv
2. 修改代码，获得新的**2-gram**语言模型
    + 进行文本清洗，获得所有的纯文本
    + 将这些文本进行切词
    + 送入之前定义的语言模型中，判断文本的合理程度

In [37]:
import jieba,re

def clean_text(string):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    return ''.join(pattern.findall(string))

def cut_text(string):
    return jieba.lcut(string)

In [38]:
filename = 'train.txt'
corpus = [clean_text(line) for line in open(filename)]
corpus[:10]

['法律要求残疾保险吗',
 '债权人可以在死后人寿保险吗',
 '旅行者保险有租赁保险吗',
 '我可以开一辆没有保险的新车吗',
 '人寿保险的现金转出价值是否应纳税',
 '如何报告年金收入',
 '家庭保险涵盖什么',
 '什么是简单的退休计划',
 '社会保险残疾保险是什么',
 '汽车保险是否预付']

In [39]:
corpus_cut = []
for i in corpus:
    corpus_cut += cut_text(i)
    
corpus_cut[:10]

['法律', '要求', '残疾', '保险', '吗', '债权人', '可以', '在', '死', '后']

In [40]:
corpus_cut_2 = [''.join(corpus_cut[i:i+2]) for i in range(len(corpus_cut[:-2]))]

corpus_cut_2[:10]

['法律要求', '要求残疾', '残疾保险', '保险吗', '吗债权人', '债权人可以', '可以在', '在死', '死后', '后人寿保险']

In [41]:
from collections import Counter
corpus_count_2 = Counter(corpus_cut_2)

corpus_count_2.most_common(10)

[('健康保险', 1349),
 ('什么是', 1153),
 ('保险是否', 975),
 ('我的', 726),
 ('残疾保险', 659),
 ('房主保险', 603),
 ('我可以', 530),
 ('保险吗', 510),
 ('是否覆盖', 504),
 ('年金', 464)]

In [42]:
corpus_count_1 = Counter(corpus_cut)
corpus_count_1.most_common(10)

[('保险', 5013),
 ('的', 3220),
 ('人寿保险', 2962),
 ('什么', 2677),
 ('吗', 2479),
 ('是', 2344),
 ('我', 2051),
 ('是否', 1862),
 ('可以', 1704),
 ('健康', 1513)]

In [43]:
# 进行拉普拉斯平滑
def prob_1_(word):
    if word in corpus_count_1:
        return (corpus_count_1[word]+1) / (len(corpus_cut)+len(corpus_count_1))
    else:
        return 1 / (len(corpus_cut)+len(corpus_count_1))

In [44]:
# 用拉普拉斯平滑，参考《Speech and Language Processing》第3版

def prob_2_(word1,word2):
    if word1 + word2 in corpus_count_2:
        prob_word12 = (corpus_count_2[word1+word2] + 1) / (corpus_count_1[word1]+len(corpus_cut))
        return prob_word12
    else:
        return 1 / (corpus_count_1[word1]+len(corpus_cut))
    
def get_prob_(sentence):
    words = cut_text(clean_text(sentence))
    sentence_pro = 1
    for i,word in enumerate(words[:-1]):
        next_word = words[i+1]
        sentence_pro *= prob_2_(word,next_word)
    return sentence_pro

In [45]:
len(corpus_count_1),len(corpus_cut)

(3478, 74308)

In [46]:
get_prob_('人寿保险的现金转出价值是否应纳税')

1.8615312489602907e-31

In [47]:
get_prob_('保险的现金人寿转出价值是否应纳税')

2.5343120949984735e-36

> 

#### 3. 获得最优质的的语言

当我们能够生成随机的语言并且能判断之后，我们就可以生成更加合理的语言了。请定义 generate_best 函数，该函数输入一个语法 + 语言模型，能够生成**n**个句子，并能选择一个最合理的句子: 



提示，要实现这个函数，你需要Python的sorted函数

In [13]:
sorted([1, 3, 5, 2])

[1, 2, 3, 5]

这个函数接受一个参数key，这个参数接受一个函数作为输入，例如

In [15]:
sorted([(2, 5), (1, 4), (5, 0), (4, 4)], key=lambda x: x[0])

[(1, 4), (2, 5), (4, 4), (5, 0)]

能够让list按照第0个元素进行排序.

In [16]:
sorted([(2, 5), (1, 4), (5, 0), (4, 4)], key=lambda x: x[1])

[(5, 0), (1, 4), (4, 4), (2, 5)]

能够让list按照第1个元素进行排序.

In [17]:
sorted([(2, 5), (1, 4), (5, 0), (4, 4)], key=lambda x: x[1], reverse=True)

[(2, 5), (1, 4), (4, 4), (5, 0)]

能够让list按照第1个元素进行排序, 但是是递减的顺序。

>

In [49]:
def generate_best(gram,target,n): 
    sent_list = []
    for i in range(n):
        sent = generate(gram,target)
        sent_list.append((sent,get_prob(sent)))
    sorted(sent_list,key=lambda x:x[1],reverse=True)
    return sent_list[0][0]

In [50]:
# 用课堂上的'article_9k.txt'训练的语言模型来计算概率
# 用PUA句子生成器生成句子，并挑选出最合理的句子

generate_best(pua_grammar,target='PUA',n=20)

'小魔女，你的裙子真美你的美人痣真美你的鼻子真好看。我是挨踢男，你想和我跳一支小苹果吗？'

好了，现在我们实现了自己的第一个AI模型，这个模型能够生成比较接近于人类的语言。

>

Q: 这个模型有什么问题？ 你准备如何提升？ 

Ans:

一是语言模型语料库还不是很大，因此有些词语的2-gram不在语料库中，会用平滑的方式来处理，合理和不合理的某些2-gram的概率会相同。解决办法的话是增加更多语料；

二是存储语言模型需要耗去大量的内存，随着语料库的增加，以及n-gram中的n增加，这种问题会更大。可以训练神经网络语言模型，训练好后不用保存切词后的语料，保存模型的参数即可，然后用模型参数就可以计算出一个句子的概率。

##### 以下内容为可选部分，对于绝大多数同学，能完成以上的项目已经很优秀了，下边的内容如果你还有精力可以试试，但不是必须的。

#### 4. (Optional) 完成基于Pattern Match的语句问答
> 我们的GitHub仓库中，有一个assignment-01-optional-pattern-match，这个难度较大，感兴趣的同学可以挑战一下。


#### 5. (Optional) 完成阿兰图灵机器智能原始论文的阅读
1. 请阅读阿兰图灵关于机器智能的原始论文：https://github.com/Computing-Intelligence/References/blob/master/AI%20%26%20Machine%20Learning/Computer%20Machinery%20and%20Intelligence.pdf 
2. 并按照GitHub仓库中的论文阅读模板，填写完毕后发送给我: mqgao@kaikeba.com 谢谢

> 

各位同学，我们已经完成了自己的第一个AI模型，大家对人工智能可能已经有了一些感觉，人工智能的核心就是，我们如何设计一个模型、程序，在外部的输入变化的时候，我们的程序不变，依然能够解决问题。人工智能是一个很大的领域，目前大家所熟知的深度学习只是其中一小部分，之后也肯定会有更多的方法提出来，但是大家知道人工智能的目标，就知道了之后进步的方向。

然后，希望大家对AI不要有恐惧感，这个并不难，大家加油！

>

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1561828422005&di=48d19c16afb6acc9180183a6116088ac&imgtype=0&src=http%3A%2F%2Fb-ssl.duitang.com%2Fuploads%2Fitem%2F201807%2F28%2F20180728150843_BECNF.thumb.224_0.jpeg)